# Instruction
Run the code below, there will be a GUI showing up.

**Entry Field**

type in commands one at a time then click Move button.

**Move Button**

click the button, program will excute the input in entry fields and update the interface

**Open Button**

click the button, you can upload sulotion files(in txt, patial or non-partial) and program will automatically update the interface per the file in uploaded.

**Exit Button**

click the button to terminate a game in progress and the log will be output automatically with name log_[datetime] under the same folder.(Note: Sometimes, the exit function will stuck in jupyter environment but it can run smoothly when in other IDE )

In [1]:
# ---------------------------- package setup ------------------------------- #
from tkinter import *
from tkinter import messagebox
from tkinter import filedialog
from datetime import datetime
import numpy as np

# ---------------------------- parameter setup ------------------------------- #
CURRENT = datetime.now()
THEME_COLOR = "#375362"
STARTING_POSITION = np.array([(1, 3), (1, 4), (1, 5),
                              (2, 3), (2, 4), (2, 5),
                              (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7),
                              (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7),
                              (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7),
                              (6, 3), (6, 4), (6, 5),
                              (7, 3), (7, 4), (7, 5)])
dot_info = {}
log = []
translation = {"a": (1, 3), "b": (1, 4), "c": (1, 5),
               "d": (2, 3), "e": (2, 4), "f": (2, 5),
               "g": (3, 1), "h": (3, 2), "i": (3, 3), "j": (3, 4), "k": (3, 5), "l": (3, 6), "m": (3, 7),
               "n": (4, 1), "o": (4, 2), "p": (4, 3), "x": (4, 4), "P": (4, 5), "O": (4, 6), "N": (4, 7),
               "M": (5, 1), "L": (5, 2), "K": (5, 3), "J": (5, 4), "I": (5, 5), "H": (5, 6), "G": (5, 7),
               "F": (6, 3), "E": (6, 4), "D": (6, 5),
               "C": (7, 3), "B": (7, 4), "A": (7, 5)}

# ---------------------------- UI setup ------------------------------- #
window = Tk()
window.title("Peg Solitaire Game")
window.config(padx=50, pady=50, background=THEME_COLOR)

for key, (x, y) in translation.items():
    globals()[f"dot{x}{y}"] = Button(text=f"🔴{key}", highlightthickness=0)
    globals()[f"dot{x}{y}"].grid(row=x, column=y)
    if x != 4 or y != 4:
        dot_info[f"dot{x}{y}"] = 1
    else:
        dot_info[f"dot{x}{y}"] = 0
globals()["dot44"].config(text="⚪x")

user_input = Entry()
user_input.insert(INSERT, "e.g. ex")
user_input.grid(row=1, column=10)

for _ in range(0, 7):
    row_label = Label(text=f"{_}", background=THEME_COLOR, foreground="white")
    row_label.grid(row=0, column=_ + 1)
    column_label = Label(text=f"{_}", background=THEME_COLOR, foreground="white")
    column_label.grid(row=_ + 1, column=0)


# ---------------------------- Test for valid input ------------------------------- #
def valid_check():
    """
    Check if user input right from/to dot_info are valid
    """

    # Get the input value from entry
    try:
        from_dot = translation[user_input.get()[0]]
        from_dot_first = int(from_dot[0])
        from_dot_second = int(from_dot[1])

        to_dot = translation[user_input.get()[1]]
        to_dot_first = int(to_dot[0])
        to_dot_second = int(to_dot[1])

        middle_dot_first = int((from_dot_first + to_dot_first) / 2)
        middle_dot_second = int((from_dot_second + to_dot_second) / 2)
        for k, (a, b) in translation.items():
            if (a, b) == (middle_dot_first, middle_dot_second):
                middle_dot = k

        valid_jump = [(from_dot_first - 2, from_dot_second), (from_dot_first + 2, from_dot_second),
                      (from_dot_first, from_dot_second - 2), (from_dot_first, from_dot_second + 2)]
    # 1. Check if the dot exist
    except KeyError:
        messagebox.showwarning(title="Warning", message="The input dot position not exist!")

    else:
        # 2. Check if the from_dot is filled and the to_dot is empty:
        if dot_info[f"dot{from_dot_first}{from_dot_second}"] != 1 or dot_info[f"dot{to_dot_first}{to_dot_second}"] != 0:
            messagebox.showwarning(title="Warning", message="Please make sure the from dot is filled and the to dot is "
                                                            "empty!")
            is_valid = False

        # 3. Check if it's a diagonal jump
        elif (to_dot_first, to_dot_second) not in valid_jump:
            messagebox.showwarning(title="Warning", message="It's a diagonal jump! Please try again!")
            is_valid = False

        # 4. Check if the middle dot is filled
        elif dot_info[f"dot{middle_dot_first}{middle_dot_second}"] != 1:
            messagebox.showwarning(title="Warning", message="The dot between these two is empty!")
            is_valid = False

        else:
            is_valid = True

        return is_valid, from_dot_first, from_dot_second, middle_dot_first, middle_dot_second, to_dot_first, \
            to_dot_second, middle_dot


# ---------------------------- Make dot move ------------------------------- #
def move():
    """
    Make and record dot move if input are valid
    """
    try:
        is_valid, from_dot_first, from_dot_second, middle_dot_first, middle_dot_second, to_dot_first, to_dot_second, \
            middle_dot = valid_check()
    except TypeError:
        is_valid = False

    if is_valid:
        # 1. Record the move in log file
        try:
            with open(f"log_{CURRENT}.txt", mode="a") as file:
                file.write(f"{user_input.get()},")
        except FileNotFoundError:
            with open(f"log_{CURRENT}.txt", mode="w") as file:
                file.write(f"{user_input.get()},")

        # 2. Update dot color
        globals()[f"dot{from_dot_first}{from_dot_second}"].config(text=f"⚪{user_input.get()[0]}")
        globals()[f"dot{middle_dot_first}{middle_dot_second}"].config(text=f"⚪{middle_dot}")
        globals()[f"dot{to_dot_first}{to_dot_second}"].config(text=f"🔴{user_input.get()[1]}")

        # 3. Update dot_info dictionary
        dot_info[f"dot{from_dot_first}{from_dot_second}"] = 0
        dot_info[f"dot{middle_dot_first}{middle_dot_second}"] = 0
        dot_info[f"dot{to_dot_first}{to_dot_second}"] = 1


move_button = Button(text="Move", command=move)
move_button.grid(row=4, column=10)


# ---------------------Open button to import solution--------------------- #
def openfile():
    filetypes = (
        ('text files', '*.txt'),
        ('All files', '*.*')
    )

    # 1. Show the open file dialog
    solution_file = filedialog.askopenfile(filetypes=filetypes)
    # 2. Read the text file and show its content on the Text
    data = solution_file.read().split(",")
    # 3. Loop
    for _ in data:
        user_input.delete(0, END)
        user_input.insert(0, f"{_}")
        move()


button = Button(text="Open", command=openfile)
button.grid(row=8, column=10)

# ---------------------Exit button to terminate the game--------------------- #
exit_button = Button(text="Exit", command=window.destroy)
exit_button.grid(row=8, column=3, columnspan=3)

window.mainloop()


2022-11-01 14:09:24.679 python[46049:7220873] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit
